<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Sesión Sincrónica Semana 2.

El objetivo de este cuaderno es cubrir las dudas, preguntas, y dificultades que detectamos en los trabajos entregados en la Semana 1 y hacer una introducción a la temática tratada en la semana 2.


**NO** es necesario editar el archivo o hacer una entrega. Los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.

# PCA

- Los datos suelen venir con información, redundancia y ruido. 


- Nuestro objetivo siempre es extraer la máxima información posible de los datos mientras reducimos el ruido e ignoramos la información redundante. 


- Al hacer esto esamos reduciendo la dimensión y es uno de los objetivo de PCA


- PCA nos permite resumir este conjunto de datos en un número pequeño de variables representativas.


- Para ello busca transformar los datos de forma tal que tengamos la máxima información posible. 


- Como se mide información? En PCA y en otrós metodos estadísticos utilizamos la varianza. 


- Si algo varía más entonces tiene más información.


- PCA identifica entonces eje que representa la mayor cantidad de variación en los datos.

- Como lo hace? A través de combinaciones lineales

## Importancia de Estandarizar: Ejemplo

In [ ]:
#Cargamos las librerías a utilizar
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb

np.set_printoptions(precision=2)

In [ ]:
import pandas as pd

# Cargamos y visualizamos la primeras observaciones de los datos
X2 = pd.read_csv('data/simulated_data1.csv')
X2.head()

In [ ]:
X2.describe()

In [ ]:
X2.cov()

In [ ]:
from pca import pca

model = pca()

results = model.fit_transform(X2, verbose=False)

# Plot explained variance
fig, ax = model.plot()

In [ ]:
#ver los pesos o loadings
loadings_pca=model.results['loadings'].transpose()
loadings_pca

In [ ]:
#PCA estandarizado
model_norm = pca( normalize=True, verbose=False)

# Fit transform
results_norm = model_norm.fit_transform(X2)

# Plot explained variance
fig, ax = model_norm.plot()

In [ ]:
X2.plot.scatter(x="X1", y="X2", c ="blue")

In [ ]:
X2_std = (X2 - X2.mean())/X2.std()

X2_std.plot.scatter(x="X1", y="X2", c ="blue")

---


## PCA como filtro de ruido: Dígitos escritos a mano

- El PCA también puede usarse como un enfoque de filtrado para datos con ruido.

- La idea es, cualquier componente con una varianza mucho mayor que el efecto del ruido debería verse relativamente poco afectado por el ruido. Por lo tanto, si reconstruyes los datos utilizando solo el subconjunto más grande de componentes principales, deberías estar conservando preferentemente la señal y eliminando el ruido.


- Veamos cómo se ve esto con los [datos de dígitos](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) disponibles en los datos de [scikit-learn](https://scikit-learn.org/stable/). 


Comenzamos cargando los datos:


In [ ]:
from sklearn.datasets import load_digits

from sklearn.decomposition import PCA


digits = load_digits()
digits.data.shape

- Vamos a graficar algunos de los datos de entrada sin ruido, teniendo en cuenta que son imágenes de 8×8 píxeles. :

In [ ]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
plot_digits(digits.data)

Ahora, agreguemos algo de ruido aleatorio para crear un conjunto de datos ruidoso (aumentamos la varianza) y volvamos a graficarlo:



In [ ]:
np.random.seed(42)

# agregamos "varianza"
noisy = np.random.normal(digits.data, 4)
plot_digits(noisy)

A simple vista, es evidente que las imágenes están ruidosas y contienen píxeles que no aportan información. 

Vamos a entrenar un PCA en los datos ruidosos, solicitando que la proyección preserve el 50% de la varianza:

In [ ]:
pca = PCA(0.50).fit(noisy)
pca.n_components_

Aquí, el 50% de la varianza equivale a 12 componentes principales. Ahora, calculamos estos componentes y luego utilizamos la inversa de la transformación para reconstruir los dígitos filtrados:



In [ ]:
components = pca.transform(noisy)
filtered = pca.inverse_transform(components)
plot_digits(filtered)

- PCA tiene la capacidad de conservar la señal importante y filtrar el ruido. Esto es especialmente útil cuando trabajamos con datos que pueden estar contaminados con ruido aleatorio.
  
- Esta propiedad convierte a PCA en una herramienta poderosa para seleccionar las características más relevantes de los datos.

- En lugar de entrenar un clasificador con datos de alta dimensionalidad (lo que puede ser costoso y propenso a errores), puedes entrenarlo usando una representación de menor dimensión generada por PCA.

- Al reducir la dimensionalidad, PCA no solo hace que el proceso de entrenamiento sea más eficiente, sino que también ayuda a eliminar el ruido aleatorio en las entradas, mejorando la precisión del clasificador.


### PCA para visualización

- Para obtener una mejor intuición sobre las relaciones entre estos puntos, podemos usar PCA para proyectarlos a un número de dimensiones más manejable, digamos dos

- Los datos consisten en imágenes de 8×8 píxeles, lo que significa que son de 64 dimensiones.

In [ ]:
pca = PCA(2)  # projectar de 64 a 2 dimensiones
projected = pca.fit_transform(digits.data)
print(digits.data.shape)
print(projected.shape)

Ahora podemos graficar los dos primeros componentes principales de cada punto para aprender más sobre los datos:


In [ ]:
plt.figure(figsize=(10, 8))  # Ajusta el tamaño de la figura, en este caso, 10x8 pulgadas

plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

plt.show()  # Muestra la figura

- Esta proyección nos permite visualizar cómo se distribuyen los dígitos en estas dos nuevas dimensiones, que son las que capturan la mayor parte de la información importante.

- Esencialmente, hemos encontrado una manera de representar la información más importante de los datos (originalmente en 64 dimensiones) en solo 2 dimensiones. Esta simplificación nos permite ver patrones y diferencias entre los dígitos de manera más clara, sin necesidad de saber de antemano cuáles son los dígitos.

- Lo hicimos de manera no supervisada, lo que significa que encontramos estas direcciones importantes sin usar ninguna información sobre qué dígito es cada punto (es decir, sin usar las etiquetas de los datos).

### ¿Qué significan los componentes?

Cuando reducimos la dimensión de los datos con PCA, los datos originales (en este caso, imágenes de dígitos) se representan en un espacio más pequeño. Pero, ¿qué significan realmente estas nuevas dimensiones?

Imagina que cada imagen está formada por 64 valores, uno para cada píxel. Estos valores se organizan en un vector llamado \( x \), que luce algo así:

$$
x = [x_1, x_2, x_3, \cdots, x_{64}]
$$

#### Construcción de la imagen

Podemos pensar en la construcción de la imagen como una combinación de estos píxeles:

- Multiplicamos cada valor del vector \( x \) por el píxel correspondiente en la imagen.
- Luego, sumamos todos esos productos para reconstruir la imagen completa.

Esto se ve así:

$$
\text{imagen}(x) = x_1 \cdot \text{(píxel 1)} + x_2 \cdot \text{(píxel 2)} + x_3 \cdot \text{(píxel 3)} + \cdots + x_{64} \cdot \text{(píxel 64)}
$$

#### Reducción de Dimensión

Ahora, si quisiéramos reducir la dimensión de estos datos, una forma de hacerlo sería poner a cero la mayoría de estos valores y dejar solo unos pocos.

Por ejemplo, si solo usamos los primeros 8 valores (correspondientes a los primeros 8 píxeles), obtendríamos una versión simplificada de la imagen en un espacio de solo 8 dimensiones. Pero, esta versión no representaría muy bien la imagen completa, porque habríamos descartado casi el 90% de los píxeles.

### ¿Por qué importa?

La reducción de dimensión con PCA no solo elige algunos píxeles al azar; en lugar de eso, encuentra las combinaciones de píxeles que mejor representan la variación en las imágenes, asegurándose de que la información más importante se conserve incluso en un espacio más pequeño.


In [ ]:
#from sklearn.datasets import load_digits
import seaborn as sns

digits = load_digits()
sns.set_style('white')



def plot_pca_components(x, coefficients=None, mean=0, components=None,
                        imshape=(8, 8), n_components=8, fontsize=12,
                        show_mean=True):
    if coefficients is None:
        coefficients = x
        
    if components is None:
        components = np.eye(len(coefficients), len(x))
        
    mean = np.zeros_like(x) + mean
        

    fig = plt.figure(figsize=(1.2 * (5 + n_components), 1.2 * 2))
    g = plt.GridSpec(2, 4 + bool(show_mean) + n_components, hspace=0.3)

    def show(i, j, x, title=None):
        ax = fig.add_subplot(g[i, j], xticks=[], yticks=[])
        ax.imshow(x.reshape(imshape), interpolation='nearest')
        if title:
            ax.set_title(title, fontsize=fontsize)

    show(slice(2), slice(2), x, "True")
    
    approx = mean.copy()
    
    counter = 2
    if show_mean:
        show(0, 2, np.zeros_like(x) + mean, r'$\mu$')
        show(1, 2, approx, r'$1 \cdot \mu$')
        counter += 1

    for i in range(n_components):
        approx = approx + coefficients[i] * components[i]
        show(0, i + counter, components[i], r'$c_{0}$'.format(i + 1))
        show(1, i + counter, approx,
             r"${0:.2f} \cdot c_{1}$".format(coefficients[i], i + 1))
        if show_mean or i > 0:
            plt.gca().text(0, 1.05, '$+$', ha='right', va='bottom',
                           transform=plt.gca().transAxes, fontsize=fontsize)

    show(slice(2), slice(-2, None), approx, "Approx")
    return fig



In [ ]:
fig = plot_pca_components(digits.data[10],
                          show_mean=False)

- **Fila Superior**: Muestra cómo contribuyen los píxeles individuales a la imagen final.
- **Fila Inferior**: Muestra cómo se acumula la contribución de esos píxeles a medida que se agregan para construir la imagen.

- Si usamos solo 8 píxeles para reconstruir la imagen, solo podremos capturar una pequeña parte de la imagen original (que tiene 64 píxeles).
- Si seguimos sumando los píxeles restantes, al final recuperaremos la imagen completa.

### Bases Alternativas y PCA

- **Representación Píxel a Píxel**: No es la única forma de construir la imagen. También podemos usar otras "funciones base", que son combinaciones predefinidas de píxeles.
- **Ejemplo**: La imagen podría construirse sumando una "imagen base" (media) más combinaciones de funciones base, así:

  $$
  \text{imagen}(x) = \text{media} + x_1 \cdot \text{(base 1)} + x_2 \cdot \text{(base 2)} + x_3 \cdot \text{(base 3)} + \cdots
  $$

#### ¿Qué Hace PCA?

- **Selección de Funciones Base**: PCA es como un proceso que elige las mejores funciones base para que, al sumar solo las primeras pocas, se pueda reconstruir la mayor parte de la información de los datos.
- **Componentes Principales**: Son los coeficientes que multiplican cada función base en la serie. Estas funciones son las que capturan las características más importantes de los datos.

- **Imagen Reconstruida**: Si reconstruimos un dígito usando la media más las primeras ocho funciones base de PCA, obtendremos una representación que ya captura gran parte de la imagen original.

In [ ]:
pca = PCA(n_components=8)
Xproj = pca.fit_transform(digits.data)
sns.set_style('white')
fig = plot_pca_components(digits.data[10], Xproj[10],
                          pca.mean_, pca.components_)


A diferencia de la base de píxeles, la base de PCA nos permite recuperar las características sobresalientes de la imagen con solo una media más ocho componentes.

Este es el sentido en el que PCA proporciona una representación de baja dimensionalidad de los datos: descubre un conjunto de funciones base que son más eficientes que la base de píxeles nativa de los datos de entrada.


---
---

# Descomposición en Valores Singulares. 

---
---


En la segunda semana  nos enfocamos en otra forma de análisis matricial que conduce a representaciones de baja dimensión de matrices altamente dimensionales.

Este enfoque llamado descomposición en valores singulares (SVD), permite una representación exacta de cualquier matriz, facilitando también la eliminación de las partes menos importantes de esa representación. 

Por supuesto, cuantas menos dimensiones elijamos, menos precisa será la aproximación.


## Usos de la SVD

- Reducción de Dimensiones: Puede usarse para reducir la dimensionalidad de los datos, manteniendo la información más importante.
- Filtrado de Ruido: Ayuda a eliminar el ruido en los datos al conservar sólo los componentes principales.
- Recomendación: En sistemas de recomendación, SVD puede usarse para predecir los elementos que un usuario podría preferir.

## Diferencias entre SVD y PCA

   - Objetivo:
     - SVD descompone una matriz en tres matrices para revelar sus propiedades matemáticas.
     - PCA busca una proyección que capture la mayor varianza en los datos.

   - Metodología:
     -  SVD se aplica directamente a la matriz de datos.
     -  PCA se aplica a la matriz de covarianza de los datos, y a menudo se utiliza SVD en el proceso de calcular PCA.

   - Aplicación:
     - SVD tiene una variedad de aplicaciones en matemáticas, estadísticas y ciencias de la computación.
     - PCA se utiliza principalmente para la visualización y reducción de dimensiones en análisis de datos.

En resumen, aunque SVD y PCA pueden parecer similares y PCA incluso puede implementarse utilizando SVD, tienen objetivos y aplicaciones diferentes. SVD es una técnica más general y fundamental, mientras que PCA es una aplicación específica de análisis de datos que a menudo utiliza SVD en su cálculo.


--- 

Iniciamos este cuaderno con una introducción general y las definiciones matemáticas necesarias. Luego exploramos la idea de reconocimiento facial aplicados a los rostros de Olivetti.


## Definiciones

- Supongamos que $ X $ es una matriz $ n \times k $  con rango $ r $. Donde necesariamente, $ r \leq \min(n,k) $. (Recordemos que el [rango de una matriz](https://es.wikipedia.org/wiki/Rango_(%C3%A1lgebra_lineal)) es el número máximo de columnas (filas respectivamente) que son linealmente independientes.)


- Podemos pensar esta matriz $X$ como los datos

    - Cada fila es una observación
    - Cada columna es una variable aleatoria que describe un atributo.


### Singular Value Decomposition

La **descomposición en valores singulares** de una matriz $X$, $ n \times k $ matrix $ X $ de rango $ r \leq \min(n,k) $ es

$$
X  = U \Sigma V^T
$$

donde

- $ U $ es una matriz $ n \times n $ cuyas columnas son eigen vectores de  $ X^T X $  

- $ V $ es una matriz $ k \times k $ cuyas columnas son eigen vectores de  $ X X^T $  

- $ \Sigma $ es una matriz $ n \times k $ en cuyos $ r $ lugares de su diagonal principal hay números positivos  $ \sigma_1, \sigma_2, \ldots, \sigma_r $ llamados **valores singulares**; y las entradas restantes de  $ \Sigma $ son todos ceros.

- Los $ r $ valores singulares son la raiz cuadrada de los eigen values de la matriz $ X X^T $ de dimension $ n \times n $ y de la matriz $ k \times k $ matrix $, X^T X $  



![](figs/svd_a3.png)



#### SVD Completa vs. Delgada (Reducida)

En la SVD completa, las dimensiones de $U$, $\Sigma$, y $V$ son $n \times n$, $n \times k$, y $k \times k$ respectivamente:

\begin{align}
\underset{n\times k}{\underbrace{X}}=\underset{n\times n}{\underbrace{U}}\underset{n\times k}{\underbrace{\Sigma}}\underset{k\times k}{\underbrace{V^T}}
\end{align}

Existe también una versión delgada (o reducida) de la SVD, que evita calcular las columnas adicionales de $U$ que se multiplican por ceros en $\Sigma$. En esta descomposición, denotada como $\hat{U}\hat{\Sigma}\hat{V}^T$:

\begin{align}
\underset{n\times k}{\underbrace{X}}=\underset{n\times r}{\underbrace{\hat{U}}}\underset{r\times r}{\underbrace{\hat{\Sigma}}}\underset{r\times k}{\underbrace{\hat{V}^T}}
\end{align}

Aquí, $U$, $\Sigma$, y $V$ se reducen a matrices de dimensiones $n \times r$, $r \times r$, y $k \times r$ respectivamente.



#### SVD y Reducción de Dimensión

Cuando trabajamos con datos de alta dimensionalidad, como grandes matrices que representan, por ejemplo, imágenes, textos o series temporales, es común que algunas de esas dimensiones (columnas) no contribuyan significativamente a la estructura de los datos. Aquí es donde entra en juego la SVD para la reducción de dimensión.

##### **Paso a Paso: Reducción de Dimensión con SVD**

1. **Descomposición**:
   - Comenzamos descomponiendo la matriz de datos $X$ en las tres matrices $U$, $\Sigma$, y $V^T$ mediante SVD:
     $$
     X = U \Sigma V^T
     $$

2. **Ordenación de Importancia**:
   - Los valores singulares en $\Sigma$ están ordenados de mayor a menor. Los primeros valores singulares corresponden a las direcciones (o "conceptos") más importantes en la matriz de datos. Los valores singulares pequeños indican direcciones que contribuyen menos a la estructura general de los datos.

3. **Selección de Componentes**:
   - Para reducir la dimensionalidad, podemos seleccionar solo los primeros $r$ valores singulares más grandes (y sus correspondientes vectores singulares en $U$ y $V^T$), y descartar el resto. Esto da lugar a una matriz $\Sigma_r$, una versión truncada de $\Sigma$, y matrices reducidas $ U_r$ y $V_r^T$.

4. **Aproximación**:
   - Con las matrices reducidas $U_r$, $\Sigma_r$, y $V_r^T$, obtenemos una nueva matriz $X_r$ que es una aproximación de $X$ pero con menos dimensiones:
     $$
     X_r = U_r \Sigma_r V_r^T
     $$
   - Esta matriz $X_r$ captura la mayor parte de la variabilidad en los datos originales $X$ pero con un número mucho menor de dimensiones.



#### **Relación entre SVD Delgada y SVD Truncada**

Ambas técnicas están diseñadas para reducir la dimensión de los datos originales, pero lo hacen en diferentes grados:

1. **SVD Delgada**:
   - Mantiene todos los componentes significativos (hasta el rango $r$) y proporciona una representación compacta que aún preserva toda la información relevante de los datos.
   - Es útil cuando queremos una representación más económica sin perder ninguna dimensión significativa.

2. **SVD Truncada**:
   - Va un paso más allá al eliminar incluso algunos componentes significativos si se considera que no son necesarios para la aplicación específica. Se enfoca en conservar solo los $k$ valores singulares más grandes, donde $k$ es un número que decidimos en función de la cantidad de información que queremos preservar.
   - Es extremadamente útil en situaciones donde se necesita una representación aún más compacta, como en la compresión de datos o en la construcción de modelos de aprendizaje automático que deben ser rápidos y eficientes.



**Ejemplo**

Supongamos que tienes un conjunto de imágenes de rostros de 100x100 píxeles (10,000 dimensiones). Aplicando:

- **SVD Delgada**: Si el rango $r$ de la matriz de imágenes es 100, la SVD delgada mantendría 100 componentes, proporcionando una reconstrucción fiel pero con una reducción en el almacenamiento (de 10,000 dimensiones a 100 dimensiones).

- **SVD Truncada**: Si decides truncar la SVD a los primeros 50 valores singulares, reducirías aún más las dimensiones a 50, pero perderías algunas características más sutiles de la imagen, aunque podrías comprimirla significativamente .

### Ejemplo: Rostros de Olivetti.

Aplicar SVD a un conjunto de imágenes permite identificar los patrones principales en las caras (como la forma general de la cara, posición de los ojos, etc.), y reducir la dimensión de la imagen original. 

Al utilizar solo los componentes principales, se puede lograr una representación compacta que sigue siendo altamente efectiva para tareas como la clasificación o identificación de rostros, mientras se ignoran detalles irrelevantes.

In [ ]:
# Librerias
from sklearn.datasets import fetch_olivetti_faces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Cargar datos
oliveti_dataset = fetch_olivetti_faces(data_home="data")

In [ ]:
oliveti_dataset.DESCR

In [ ]:
oliveti_dataset.images

In [ ]:
n_samples, h, w = oliveti_dataset.images.shape

In [ ]:
#Aplasto los datos para poder trabajar "mas intuitivamente" con las matrices
X = oliveti_dataset.data
X = pd.DataFrame(X)

y = oliveti_dataset.target
y = pd.DataFrame(y, columns=['pid']) # pid is person_id

df = y.join(X)
df.head()

In [ ]:
X.describe()

In [ ]:
# Choose a figure to plot
img1=X.iloc[300].to_numpy().reshape(h,w)
plt.imshow(img1, cmap=plt.cm.gray)

In [ ]:
faces_tplot=df.drop(columns=['pid'])
n_row=1
n_col=10

plt.figure(figsize=(1.5 * n_col, 2.2 * n_row))
plt.subplots_adjust(0.6, 0.5, 1.5, 1.5)
for i in range(n_row * n_col):
    plt.subplot(n_row, n_col, i + 1)
    plt.imshow(faces_tplot.iloc[i].to_numpy().reshape((h, w)), cmap=plt.cm.gray)
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

#### 3.1. Rostro promedio. 

 Encuentre el rostro promedio, para ello calcule la media por columnas o la media por píxel y grafíquelo.

- El objetivo es  encontrar las características que hacen que los individuos sean diferentes a los demás. 

-  La razón por la que esto es necesario es porque queremos crear un sistema que pueda representar cualquier rostro. 

- Por lo tanto, calculamos los elementos que todas las caras tienen en común (la media). 

- Si extraemos esta media de las imagen, se aprecian las características que distinguen cada fotografía del resto del conjunto. Maximizamos la varianza!

In [ ]:
# Utilice este espacio para escribir el código.
Xmean = df.drop(columns=[ 'pid']).mean(axis=0)

In [ ]:
#plot the average face 
plt.imshow(Xmean.to_numpy().reshape((h, w)), cmap=plt.cm.gray)

#### 3.2. Reste el rostro promedio.

A cada una de las imágenes substraigale el rostro promedio.

In [ ]:

Demean=df.drop(columns=['pid'])-Xmean 

In [ ]:
#Some examples
n_row=8
n_col=5

plt.figure(figsize=(1.5 * n_col, 2.2 * n_row))
plt.subplots_adjust(0.6, 0.5, 1.5, 1.5)
for i in range(n_row * n_col):
    plt.subplot(n_row, n_col, i + 1)
    plt.imshow(Demean.iloc[i].to_numpy().reshape((h, w)), cmap=plt.cm.gray)
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

#### 3.3. Descomposición en Valores Singulares 

Aplique la Descomposición en Valores Singulares a estas nuevas imágenes y retenga solo $K$ eigen vectores que mejor representen las imágenes. Justifique su elección.

In [ ]:
from scipy.linalg import svd #pueden explorar con sklearn
U,S,Vt = svd(X)

In [ ]:
# observe cuan rapidamente caen los SVD
idx = range(len(S))
func = [S[0]/((i+1) ** 2) for i in idx ]

plt.figure(figsize = (8, 6))
plt.plot(idx, func, color = 'r')
plt.scatter(idx, S)

In [ ]:
rg =50
idx = range(0,rg)
var_explained = np.round(S**2/np.sum(S**2), decimals=6)

cumsum=var_explained[0:rg].cumsum()

plt.figure(figsize = (8, 6))
plt.plot(idx,cumsum , color = 'r')


In [ ]:
sum(var_explained[0:rg])

In [ ]:
n,k=X .shape

S2= np.resize(S,[n,1])*np.eye(n,k) #ponemos los valores singulares en una matriz diagonal

l = 10

reconstructed=np.dot(U[:,0:l],np.dot(S2[0:l,0:l],Vt[0:l,:]))
reconstructed = pd.DataFrame(reconstructed)
reconstructed

In [ ]:
imag10_index=df.loc[df['pid'] == 10].index
imag10=reconstructed.iloc[imag10_index]

imag10

In [ ]:
# Graficamos
n_row=2
n_col=5

plt.figure(figsize=(1.5 * n_col, 2.2 * n_row))
plt.subplots_adjust(0.6, 0.5, 1.5, 1.5)
for i in range(n_row * n_col):
    plt.subplot(n_row, n_col, i + 1)
    plt.imshow(imag10.iloc[i].to_numpy().reshape((h, w)), cmap=plt.cm.gray)
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

# Información de Sesión

In [ ]:
import session_info

session_info.show(html=False)